# Using neural classification
As has been proven by [Wang (2017)](https://arxiv.org/abs/1705.00648), neural classifiers carry better results than non-neural classifiers when detecting fake news. However, it is unknown how well neural networks classify fake news when using previously mentioned text embeddings. 
In this notebook, the second research question will be answered: *how well do neural network architecture classify fake news compared to non-neural classification algorithms?*

<hr>

In [1]:
# General imports
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import keras
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Reshape, Conv1D, Flatten

# Set offline mode for plotly
init_notebook_mode(connected = True)

# The DataLoader class gives access to pretrained vectors from the Liar dataset
from data_loader import DataLoader
data = DataLoader()

Using TensorFlow backend.


In [2]:
general = data.get_dfs()

# Recode labels from 6 to 3
def recode(label):
    if label == 'false' or label == 'pants-fire' or label == 'barely-true':
        return 0
    elif label == 'true' or label == 'mostly-true':
        return 2
    elif label == 'half-true':
        return 1

for dataset in general.keys():
    general[dataset]['label'] = general[dataset]['label'].apply(lambda label: recode(label))

<hr>

## Bidirectional LSTMs

In [3]:
bert = data.get_bert()

In [3]:
# Get max-pooled BERT embeddings from RQ1
def max_pool(statement):
    if len(statement) > 1:
        return [row.max() for row in np.transpose([[token_row.max() for token_row in np.transpose(np.array(sentence))] for sentence in statement])]
    else:
        return [token_row.max() for token_row in np.transpose(statement[0])]

max_pooled_bert = {
    dataset: pd.DataFrame(list(bert[dataset].statement.apply(lambda statement: max_pool(statement)).values))
    for dataset in bert.keys()
}

In [3]:
def get_bilstm_score(X_train, X_test, X_validation, y_train = general['train']['label'], y_test = general['test']['label'], y_validation = general['validation']['label'], reshape = True):
    # Rearrange data types    
    params = locals().copy()    
    inputs = {
        dataset: np.array(params[dataset])
        for dataset in params.keys()
    }
    
    for dataset in inputs.keys():
        if dataset[0:1] == 'X' and reshape:
            # Reshape datasets from 2D to 3D
            inputs[dataset] = np.reshape(inputs[dataset], (inputs[dataset].shape[0], inputs[dataset].shape[1], 1))
        elif dataset[0:1] == 'y':
            inputs[dataset] = np_utils.to_categorical(np.array(inputs[dataset]), 3)
    
    # Set model parameters
    epochs = 5
    batch_size = 32
    input_shape = X_train.shape

    # Create the model
    model = Sequential()
    model.add(Bidirectional(LSTM(64, input_shape = input_shape)))
    model.add(Dropout(0.8))
    model.add(Dense(3, activation = 'softmax'))
    model.compile('sgd', 'categorical_crossentropy', metrics = ['accuracy']) 
    
    # Fit the training set over the model and correct on the validation set
    model.fit(inputs['X_train'], inputs['y_train'],
            batch_size = batch_size,
            epochs = epochs,
            validation_data = (inputs['X_validation'], inputs['y_validation']))
    
    # Get score over the test set
    score, acc = model.evaluate(inputs['X_test'], inputs['y_test'])
    
    return acc

In [5]:
get_bilstm_score(max_pooled_bert['train'], max_pooled_bert['test'], max_pooled_bert['validation'])

Instructions for updating:
Colocations handled automatically by placer.
2019-05-21 12:40:37,047 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-05-21 12:40:37,448 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-05-21 12:40:37,537 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

0.43715415052745654

Apparently, the condensed datasets from RQ1 do not perform well when using a neural classifier. The next step is trying out a padding approach.

In [65]:
%%time

# Store accuracies
accuracies = {
    padding_len: 0.0 for padding_len in list(range(5,36))
}

concatenated_bert = {
    dataset: [np.concatenate(np.array(statement)) for statement in bert[dataset].statement]
    for dataset in bert.keys()
}

for max_len in accuracies.keys():
    padded_bert = {
        dataset: sequence.pad_sequences(concatenated_bert[dataset], maxlen = max_len, dtype = float)
        for dataset in concatenated_bert.keys()
    }
    
    accuracies[max_len] = get_bilstm_score(padded_bert['train'], padded_bert['test'], padded_bert['validation'], reshape = False)

Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 69s 7ms/step - loss: 1.1184 - acc: 0.4145 - val_loss: 1.0170 - val_acc: 0.5093
Epoch 2/5
10235/10235 [==============================] - 17s 2ms/step - loss: 1.0597 - acc: 0.4532 - val_loss: 1.0196 - val_acc: 0.5031
Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0432 - acc: 0.4707 - val_loss: 1.0158 - val_acc: 0.5125
Epoch 4/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0362 - acc: 0.4775 - val_loss: 1.0134 - val_acc: 0.5117
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 46s 4ms/step - loss: 1.1094 - acc: 0.4181 - val_loss: 1.0184 - val_acc: 0.4907
Epoch 2/5
10235/10235 [==============================] - 18s 2ms/step - loss: 1.0558 - acc: 0.4556 - val_loss: 1.0125 - val_acc: 0.5125
Epoch 3/5
10235/102

10235/10235 [==============================] - 31s 3ms/step - loss: 1.0350 - acc: 0.4852 - val_loss: 1.0104 - val_acc: 0.5101
Epoch 4/5
10235/10235 [==============================] - 30s 3ms/step - loss: 1.0312 - acc: 0.4890 - val_loss: 1.0051 - val_acc: 0.5140
Epoch 5/5
1265/1265 [==============================] - 3s 3ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 91s 9ms/step - loss: 1.1168 - acc: 0.4210 - val_loss: 1.0127 - val_acc: 0.5047
Epoch 2/5
10235/10235 [==============================] - 37s 4ms/step - loss: 1.0508 - acc: 0.4668 - val_loss: 1.0166 - val_acc: 0.5086
Epoch 3/5
10235/10235 [==============================] - 33s 3ms/step - loss: 1.0364 - acc: 0.4793 - val_loss: 1.0032 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============================] - 32s 3ms/step - loss: 1.0281 - acc: 0.4893 - val_loss: 1.0000 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 4s 3ms/step
Train on 10

In [67]:
bert_rounds

[{5: 0.513043478637816,
  6: 0.5122529648038239,
  7: 0.5075098817998712,
  8: 0.5083003953747127,
  9: 0.49407114631573673,
  10: 0.5098814233018476,
  11: 0.507509881752753,
  12: 0.5035573126299108,
  13: 0.4996047434599503,
  14: 0.5027667987488004,
  15: 0.507509881752753,
  16: 0.5090909091144682,
  17: 0.5067193679187609,
  18: 0.5114624509227135,
  19: 0.5098814233018476,
  20: 0.5162055336203971,
  21: 0.5193675893097527,
  22: 0.5193675893097527,
  23: 0.5233201584797131,
  24: 0.5217391304583417,
  25: 0.5106719371358397,
  26: 0.5106719367824524,
  27: 0.5177865616417685,
  28: 0.5075098817998712,
  29: 0.513043478637816,
  30: 0.5043478264639029,
  31: 0.5162055336203971,
  32: 0.5075098817998712,
  33: 0.5114624509698318,
  34: 0.513833992471808,
  35: 0.5075098817998712},
 {5: 0.5098814233018476,
  6: 0.5098814229484603,
  7: 0.49723320160458684,
  8: 0.5090909091144682,
  9: 0.5051383399445077,
  10: 0.505928854131887,
  11: 0.498814229272571,
  12: 0.5114624509698318,


In [69]:
traces = [round1, round2, round3, round4, round5]

# Create traces
bert_trace = go.Scatter(
    x = list(round1.keys()),
    y = list(round1.values()),
    mode = 'lines+markers',
    name = 'BERT'
)

def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded BERT dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [3]:
elmo = data.get_elmo()

In [6]:
def calculate_round(dataset):
    # Store accuracies
    accuracies = {
        padding_len: 0.0 for padding_len in list(range(5,36))
    }

    for max_len in accuracies.keys():
        padded_dataset = {
            fold: sequence.pad_sequences(dataset[fold], maxlen = max_len, dtype = float)
            for fold in dataset.keys()
        }

        accuracies[max_len] = get_bilstm_score(padded_dataset['train'], padded_dataset['test'], padded_dataset['validation'], reshape = False)

    return accuracies

In [7]:
%%time

concatenated_elmo = {
    fold: [np.concatenate(np.array(statement)) for statement in elmo[fold]['statement']]
    for fold in elmo.keys()
}

elmo_rounds = [calculate_round(concatenated_elmo) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-04 08:49:28,085 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-04 08:49:29,218 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-04 08:49:29,336 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

10235/10235 [==============================] - 34s 3ms/step - loss: 1.1207 - acc: 0.4320 - val_loss: 1.0211 - val_acc: 0.5016
Epoch 2/5
10235/10235 [==============================] - 31s 3ms/step - loss: 1.0517 - acc: 0.4702 - val_loss: 1.0160 - val_acc: 0.5101
Epoch 3/5
10235/10235 [==============================] - 31s 3ms/step - loss: 1.0308 - acc: 0.4884 - val_loss: 1.0031 - val_acc: 0.5101
Epoch 4/5
10235/10235 [==============================] - 31s 3ms/step - loss: 1.0219 - acc: 0.4962 - val_loss: 1.0021 - val_acc: 0.5132
Epoch 5/5
1265/1265 [==============================] - 1s 1ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 38s 4ms/step - loss: 1.1081 - acc: 0.4299 - val_loss: 1.0038 - val_acc: 0.5148
Epoch 2/5
10235/10235 [==============================] - 34s 3ms/step - loss: 1.0480 - acc: 0.4731 - val_loss: 1.0029 - val_acc: 0.5140
Epoch 3/5
10235/10235 [==============================] - 35s 3ms/step - loss: 

1265/1265 [==============================] - 8s 7ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 112s 11ms/step - loss: 1.0812 - acc: 0.4300 - val_loss: 1.0226 - val_acc: 0.5125
Epoch 2/5
10235/10235 [==============================] - 89s 9ms/step - loss: 1.0550 - acc: 0.4587 - val_loss: 1.0114 - val_acc: 0.5132
Epoch 3/5
10235/10235 [==============================] - 73s 7ms/step - loss: 1.0377 - acc: 0.4820 - val_loss: 1.0068 - val_acc: 0.5109
Epoch 4/5
10235/10235 [==============================] - 74s 7ms/step - loss: 1.0326 - acc: 0.4815 - val_loss: 1.0002 - val_acc: 0.5210
Epoch 5/5
1265/1265 [==============================] - 10s 8ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 114s 11ms/step - loss: 1.0779 - acc: 0.4358 - val_loss: 1.0203 - val_acc: 0.5125
Epoch 2/5
10235/10235 [==============================] - 94s 9ms/step - loss: 1.0488 - acc: 0

In [15]:
elmo_rounds

[{5: 0.5177865616417685,
  6: 0.5241106722665869,
  7: 0.5114624509698318,
  8: 0.5067193676124919,
  9: 0.5067193679187609,
  10: 0.5059288540847687,
  11: 0.49802371574484783,
  12: 0.5011857710808162,
  13: 0.49169960507291105,
  14: 0.49723320191085574,
  15: 0.4932806327408953,
  16: 0.5169960478077764,
  17: 0.5154150200455556,
  18: 0.513043478637816,
  19: 0.5083003955396268,
  20: 0.5185770754286423,
  21: 0.5067193679187609,
  22: 0.5146245063058001,
  23: 0.5011857710808162,
  24: 0.5106719370887214,
  25: 0.5169960477606581,
  26: 0.5193675893097527,
  27: 0.5114624509227135,
  28: 0.5130434785906977,
  29: 0.5241106723137052,
  30: 0.5169960477606581,
  31: 0.5169960477606581,
  32: 0.5146245063058001,
  33: 0.5185770754286423,
  34: 0.5051383402507766,
  35: 0.5035573125827925},
 {5: 0.5122529647567056,
  6: 0.5169960478077764,
  7: 0.5146245062586818,
  8: 0.5233201584325948,
  9: 0.5138339924246897,
  10: 0.4996047434599503,
  11: 0.49090909123891896,
  12: 0.4909090912

In [14]:
traces = elmo_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded ELMo dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [12]:
%%time

transformerxl_rounds = [calculate_round(concatenated_transformerxl) for round in range(5)]

Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 14s 1ms/step - loss: 1.0872 - acc: 0.3999 - val_loss: 1.0340 - val_acc: 0.4836
Epoch 2/5
10235/10235 [==============================] - 6s 553us/step - loss: 1.0666 - acc: 0.4330 - val_loss: 1.0242 - val_acc: 0.5008
Epoch 3/5
10235/10235 [==============================] - 6s 563us/step - loss: 1.0573 - acc: 0.4496 - val_loss: 1.0170 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 6s 563us/step - loss: 1.0508 - acc: 0.4609 - val_loss: 1.0161 - val_acc: 0.5039
Epoch 5/5
1265/1265 [==============================] - 0s 267us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 14s 1ms/step - loss: 1.0840 - acc: 0.4108 - val_loss: 1.0314 - val_acc: 0.4860
Epoch 2/5
10235/10235 [==============================] - 6s 586us/step - loss: 1.0624 - acc: 0.4487 - val_loss: 1.0226 - val_acc: 0.4992
Epoch 3/5
102

10235/10235 [==============================] - 15s 1ms/step - loss: 1.0522 - acc: 0.4574 - val_loss: 1.0185 - val_acc: 0.5156
Epoch 5/5
1265/1265 [==============================] - 1s 708us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 28s 3ms/step - loss: 1.0773 - acc: 0.4165 - val_loss: 1.0360 - val_acc: 0.4883
Epoch 2/5
10235/10235 [==============================] - 15s 1ms/step - loss: 1.0589 - acc: 0.4489 - val_loss: 1.0265 - val_acc: 0.4938
Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0528 - acc: 0.4490 - val_loss: 1.0209 - val_acc: 0.5023
Epoch 4/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0476 - acc: 0.4656 - val_loss: 1.0170 - val_acc: 0.5093
Epoch 5/5
1265/1265 [==============================] - 1s 919us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 28s 3ms/step - loss: 1.0800 - acc: 0.

10235/10235 [==============================] - 11s 1ms/step - loss: 1.0677 - acc: 0.4385 - val_loss: 1.0190 - val_acc: 0.5187
Epoch 3/5
10235/10235 [==============================] - 10s 1ms/step - loss: 1.0584 - acc: 0.4524 - val_loss: 1.0123 - val_acc: 0.5280
Epoch 4/5
10235/10235 [==============================] - 11s 1ms/step - loss: 1.0469 - acc: 0.4668 - val_loss: 1.0093 - val_acc: 0.5257
Epoch 5/5
1265/1265 [==============================] - 1s 539us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 28s 3ms/step - loss: 1.0939 - acc: 0.3998 - val_loss: 1.0315 - val_acc: 0.5031
Epoch 2/5
10235/10235 [==============================] - 11s 1ms/step - loss: 1.0625 - acc: 0.4452 - val_loss: 1.0232 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 11s 1ms/step - loss: 1.0603 - acc: 0.4498 - val_loss: 1.0150 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 11s 1ms/step - loss

1265/1265 [==============================] - 2s 1ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 28s 3ms/step - loss: 1.0831 - acc: 0.4091 - val_loss: 1.0285 - val_acc: 0.4914
Epoch 2/5
10235/10235 [==============================] - 6s 634us/step - loss: 1.0634 - acc: 0.4335 - val_loss: 1.0223 - val_acc: 0.4907
Epoch 3/5
10235/10235 [==============================] - 6s 629us/step - loss: 1.0562 - acc: 0.4504 - val_loss: 1.0180 - val_acc: 0.4977
Epoch 4/5
10235/10235 [==============================] - 7s 654us/step - loss: 1.0491 - acc: 0.4616 - val_loss: 1.0148 - val_acc: 0.5070
Epoch 5/5
1265/1265 [==============================] - 0s 324us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 28s 3ms/step - loss: 1.0916 - acc: 0.4017 - val_loss: 1.0304 - val_acc: 0.5062
Epoch 2/5
10235/10235 [==============================] - 7s 702us/step - loss: 1.0671 - acc: 

10235/10235 [==============================] - 16s 2ms/step - loss: 1.0457 - acc: 0.4611 - val_loss: 1.0121 - val_acc: 0.5117
Epoch 5/5
1265/1265 [==============================] - 1s 999us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0827 - acc: 0.4063 - val_loss: 1.0299 - val_acc: 0.4922
Epoch 2/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0660 - acc: 0.4348 - val_loss: 1.0217 - val_acc: 0.5055
Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0561 - acc: 0.4497 - val_loss: 1.0161 - val_acc: 0.5101
Epoch 4/5
10235/10235 [==============================] - 17s 2ms/step - loss: 1.0468 - acc: 0.4668 - val_loss: 1.0114 - val_acc: 0.5140
Epoch 5/5
1265/1265 [==============================] - 1s 845us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 44s 4ms/step - loss: 1.0843 - acc: 0.

10235/10235 [==============================] - 11s 1ms/step - loss: 1.0538 - acc: 0.4595 - val_loss: 1.0136 - val_acc: 0.5086
Epoch 5/5
1265/1265 [==============================] - 1s 584us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0803 - acc: 0.4150 - val_loss: 1.0278 - val_acc: 0.5031TA: 1:59 - loss: 1.0948 -  - ETA: 1:22
Epoch 2/5
10235/10235 [==============================] - 12s 1ms/step - loss: 1.0605 - acc: 0.4460 - val_loss: 1.0207 - val_acc: 0.5249: 1 - ETA: 2s - loss: 1.0 - ETA: 1s 
Epoch 3/5
10235/10235 [==============================] - 12s 1ms/step - loss: 1.0543 - acc: 0.4579 - val_loss: 1.0174 - val_acc: 0.5109 8s - loss: 1.0526 - - ETA: 7s - los - ETA: 6s - loss: 1. - ETA: 4s - loss: 1.0569 - a  - ETA: 1s - loss: 1.0533 - acc: 0 - ETA: 1s - loss: 1
Epoch 4/5
10235/10235 [==============================] - 12s 1ms/step - loss: 1.0470 - acc: 0.4653 - val_loss: 1.0090 - val_acc: 0.51

Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0549 - acc: 0.4559 - val_loss: 1.0240 - val_acc: 0.4805
Epoch 4/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0476 - acc: 0.4659 - val_loss: 1.0200 - val_acc: 0.4852
Epoch 5/5
1265/1265 [==============================] - 1s 832us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 51s 5ms/step - loss: 1.0820 - acc: 0.4090 - val_loss: 1.0389 - val_acc: 0.4805
Epoch 2/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0637 - acc: 0.4387 - val_loss: 1.0330 - val_acc: 0.4961
Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0553 - acc: 0.4542 - val_loss: 1.0264 - val_acc: 0.5039
Epoch 4/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0511 - acc: 0.4607 - val_loss: 1.0211 - val_acc: 0.5055
Epoch 5/5
1265/1265 [==============================] - 1s 860us/st

1265/1265 [==============================] - 1s 579us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 49s 5ms/step - loss: 1.0882 - acc: 0.4040 - val_loss: 1.0298 - val_acc: 0.4945
Epoch 2/5
10235/10235 [==============================] - 12s 1ms/step - loss: 1.0679 - acc: 0.4375 - val_loss: 1.0174 - val_acc: 0.5062
Epoch 3/5
10235/10235 [==============================] - 12s 1ms/step - loss: 1.0560 - acc: 0.4543 - val_loss: 1.0140 - val_acc: 0.5055ETA: 5s  - ETA: 3s - loss: 1.
Epoch 4/5
10235/10235 [==============================] - 12s 1ms/step - loss: 1.0494 - acc: 0.4612 - val_loss: 1.0065 - val_acc: 0.5156
Epoch 5/5
1265/1265 [==============================] - 1s 610us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 50s 5ms/step - loss: 1.0908 - acc: 0.4089 - val_loss: 1.0296 - val_acc: 0.4984
Epoch 2/5
10235/10235 [==============================] - 12s 1ms/

10235/10235 [==============================] - 22s 2ms/step - loss: 1.0530 - acc: 0.4514 - val_loss: 1.0234 - val_acc: 0.4992
Epoch 4/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0516 - acc: 0.4546 - val_loss: 1.0193 - val_acc: 0.5086
Epoch 5/5
1265/1265 [==============================] - 1s 1ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 74s 7ms/step - loss: 1.0791 - acc: 0.4097 - val_loss: 1.0350 - val_acc: 0.4868
Epoch 2/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0628 - acc: 0.4352 - val_loss: 1.0289 - val_acc: 0.5070
Epoch 3/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0518 - acc: 0.4520 - val_loss: 1.0221 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0512 - acc: 0.4537 - val_loss: 1.0158 - val_acc: 0.5195
Epoch 5/5
1265/1265 [==============================] - 2s 1ms/step
CPU times: 

In [13]:
transformerxl_rounds

[{5: 0.5003952572468241,
  6: 0.474308300725085,
  7: 0.4996047434599503,
  8: 0.5106719370416031,
  9: 0.49802371574484783,
  10: 0.4853754944009743,
  11: 0.47905138372903755,
  12: 0.49249011895402145,
  13: 0.49249011895402145,
  14: 0.49090909128603727,
  15: 0.4940711466220057,
  16: 0.49169960507291105,
  17: 0.4893280635709348,
  18: 0.4956521739366026,
  19: 0.4940711466220057,
  20: 0.49802371579196614,
  21: 0.49802371574484783,
  22: 0.49486166040887947,
  23: 0.4948616604559978,
  24: 0.4988142296259582,
  25: 0.49723320191085574,
  26: 0.4877470359029506,
  27: 0.49644268807686365,
  28: 0.4940711465748874,
  29: 0.49486166040887947,
  30: 0.49090909123891896,
  31: 0.4735177868910929,
  32: 0.4924901189069032,
  33: 0.4869565220689585,
  34: 0.4877470358558323,
  35: 0.49802371579196614},
 {5: 0.49090909123891896,
  6: 0.4845849802607133,
  7: 0.48616600792869746,
  8: 0.49565217428998987,
  9: 0.49169960512002936,
  10: 0.49090909123891896,
  11: 0.5019762849619266,
  1

In [14]:
traces = transformerxl_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded Transformer-XL dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [5]:
flair = data.get_flair()

In [7]:
%%time

concatenated_flair = {
    fold: [np.concatenate(np.array(statement)) for statement in flair[fold]['statement']]
    for fold in flair.keys()
}

flair_rounds = [calculate_round(concatenated_flair) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-13 20:08:53,109 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-13 20:08:54,567 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-13 20:08:55,021 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

10235/10235 [==============================] - 46s 5ms/step - loss: 1.0727 - acc: 0.4228 - val_loss: 1.0356 - val_acc: 0.4969
Epoch 2/5
10235/10235 [==============================] - 35s 3ms/step - loss: 1.0522 - acc: 0.4590 - val_loss: 1.0278 - val_acc: 0.4969
Epoch 3/5
10235/10235 [==============================] - 35s 3ms/step - loss: 1.0477 - acc: 0.4678 - val_loss: 1.0164 - val_acc: 0.5031
Epoch 4/5
10235/10235 [==============================] - 35s 3ms/step - loss: 1.0410 - acc: 0.4739 - val_loss: 1.0156 - val_acc: 0.5000
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 52s 5ms/step - loss: 1.0661 - acc: 0.4329 - val_loss: 1.0256 - val_acc: 0.4907
Epoch 2/5
10235/10235 [==============================] - 38s 4ms/step - loss: 1.0515 - acc: 0.4691 - val_loss: 1.0147 - val_acc: 0.5047
Epoch 3/5
10235/10235 [==============================] - 38s 4ms/step - loss: 

1265/1265 [==============================] - 11s 8ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0662 - acc: 0.4265 - val_loss: 1.0314 - val_acc: 0.5023
Epoch 2/5
10235/10235 [==============================] - 103s 10ms/step - loss: 1.0521 - acc: 0.4462 - val_loss: 1.0248 - val_acc: 0.5086
Epoch 3/5
10235/10235 [==============================] - 90s 9ms/step - loss: 1.0467 - acc: 0.4631 - val_loss: 1.0153 - val_acc: 0.5125
Epoch 4/5
10235/10235 [==============================] - 86s 8ms/step - loss: 1.0430 - acc: 0.4677 - val_loss: 1.0158 - val_acc: 0.5109
Epoch 5/5
1265/1265 [==============================] - 11s 9ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 134s 13ms/step - loss: 1.0646 - acc: 0.4357 - val_loss: 1.0242 - val_acc: 0.5132
Epoch 2/5
10235/10235 [==============================] - 109s 11ms/step - loss: 1.0479 - a

10235/10235 [==============================] - 55s 5ms/step - loss: 1.0467 - acc: 0.4590 - val_loss: 1.0154 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 55s 5ms/step - loss: 1.0439 - acc: 0.4744 - val_loss: 1.0146 - val_acc: 0.5195
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 102s 10ms/step - loss: 1.0660 - acc: 0.4313 - val_loss: 1.0307 - val_acc: 0.4836
Epoch 2/5
10235/10235 [==============================] - 74s 7ms/step - loss: 1.0515 - acc: 0.4547 - val_loss: 1.0283 - val_acc: 0.5070
Epoch 3/5
10235/10235 [==============================] - 57s 6ms/step - loss: 1.0456 - acc: 0.4636 - val_loss: 1.0170 - val_acc: 0.5148
Epoch 4/5
10235/10235 [==============================] - 57s 6ms/step - loss: 1.0421 - acc: 0.4673 - val_loss: 1.0120 - val_acc: 0.5140
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
Train on 

10235/10235 [==============================] - 67s 7ms/step - loss: 1.0651 - acc: 0.4373 - val_loss: 1.0255 - val_acc: 0.5055
Epoch 2/5
10235/10235 [==============================] - 38s 4ms/step - loss: 1.0511 - acc: 0.4601 - val_loss: 1.0177 - val_acc: 0.5070
Epoch 3/5
10235/10235 [==============================] - 37s 4ms/step - loss: 1.0432 - acc: 0.4741 - val_loss: 1.0134 - val_acc: 0.5093
Epoch 4/5
10235/10235 [==============================] - 37s 4ms/step - loss: 1.0390 - acc: 0.4793 - val_loss: 1.0143 - val_acc: 0.5062
Epoch 5/5
1265/1265 [==============================] - 4s 3ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 77s 7ms/step - loss: 1.0674 - acc: 0.4337 - val_loss: 1.0259 - val_acc: 0.4883
Epoch 2/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0490 - acc: 0.4640 - val_loss: 1.0148 - val_acc: 0.4945
Epoch 3/5
10235/10235 [==============================] - 40s 4ms/step - loss: 

1265/1265 [==============================] - 12s 9ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 156s 15ms/step - loss: 1.0641 - acc: 0.4269 - val_loss: 1.0335 - val_acc: 0.5016
Epoch 2/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0511 - acc: 0.4468 - val_loss: 1.0205 - val_acc: 0.4961
Epoch 3/5
10235/10235 [==============================] - 91s 9ms/step - loss: 1.0475 - acc: 0.4570 - val_loss: 1.0149 - val_acc: 0.5093
Epoch 4/5
10235/10235 [==============================] - 87s 9ms/step - loss: 1.0423 - acc: 0.4685 - val_loss: 1.0136 - val_acc: 0.5039
Epoch 5/5
1265/1265 [==============================] - 11s 9ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 156s 15ms/step - loss: 1.0627 - acc: 0.4311 - val_loss: 1.0303 - val_acc: 0.4992
Epoch 2/5
10235/10235 [==============================] - 119s 12ms/step - loss: 1.0510 - a

10235/10235 [==============================] - 56s 5ms/step - loss: 1.0414 - acc: 0.4688 - val_loss: 1.0107 - val_acc: 0.5086
Epoch 4/5
10235/10235 [==============================] - 56s 5ms/step - loss: 1.0408 - acc: 0.4745 - val_loss: 1.0058 - val_acc: 0.5148
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0655 - acc: 0.4343 - val_loss: 1.0218 - val_acc: 0.4883
Epoch 2/5
10235/10235 [==============================] - 72s 7ms/step - loss: 1.0524 - acc: 0.4453 - val_loss: 1.0167 - val_acc: 0.5132
Epoch 3/5
10235/10235 [==============================] - 59s 6ms/step - loss: 1.0472 - acc: 0.4607 - val_loss: 1.0092 - val_acc: 0.5132
Epoch 4/5
10235/10235 [==============================] - 61s 6ms/step - loss: 1.0403 - acc: 0.4672 - val_loss: 1.0063 - val_acc: 0.5125
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
Train on 

1265/1265 [==============================] - 4s 3ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 103s 10ms/step - loss: 1.0666 - acc: 0.4277 - val_loss: 1.0225 - val_acc: 0.5062
Epoch 2/5
10235/10235 [==============================] - 45s 4ms/step - loss: 1.0471 - acc: 0.4655 - val_loss: 1.0131 - val_acc: 0.5171
Epoch 3/5
10235/10235 [==============================] - 38s 4ms/step - loss: 1.0434 - acc: 0.4729 - val_loss: 1.0177 - val_acc: 0.5000
Epoch 4/5
10235/10235 [==============================] - 38s 4ms/step - loss: 1.0377 - acc: 0.4797 - val_loss: 1.0060 - val_acc: 0.5109
Epoch 5/5
1265/1265 [==============================] - 4s 3ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 106s 10ms/step - loss: 1.0676 - acc: 0.4260 - val_loss: 1.0328 - val_acc: 0.4899
Epoch 2/5
10235/10235 [==============================] - 48s 5ms/step - loss: 1.0500 - acc: 0.

10235/10235 [==============================] - 87s 9ms/step - loss: 1.0464 - acc: 0.4638 - val_loss: 1.0158 - val_acc: 0.5109
Epoch 4/5
10235/10235 [==============================] - 84s 8ms/step - loss: 1.0408 - acc: 0.4721 - val_loss: 1.0101 - val_acc: 0.5101
Epoch 5/5
1265/1265 [==============================] - 11s 9ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 185s 18ms/step - loss: 1.0663 - acc: 0.4155 - val_loss: 1.0333 - val_acc: 0.4875
Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0545 - acc: 0.4381 - val_loss: 1.0271 - val_acc: 0.4969
Epoch 3/5
10235/10235 [==============================] - 95s 9ms/step - loss: 1.0486 - acc: 0.4590 - val_loss: 1.0217 - val_acc: 0.5023
Epoch 4/5
10235/10235 [==============================] - 92s 9ms/step - loss: 1.0440 - acc: 0.4680 - val_loss: 1.0127 - val_acc: 0.5086
Epoch 5/5
1265/1265 [==============================] - 12s 9ms/step
CPU t

In [8]:
flair_rounds

[{5: 0.4901185771457763,
  6: 0.49090909093265006,
  7: 0.49644268777059475,
  8: 0.4940711462686184,
  9: 0.4885375494306738,
  10: 0.4996047431065631,
  11: 0.4956521739366026,
  12: 0.49328063243462633,
  13: 0.4877470359029506,
  14: 0.4940711465748874,
  15: 0.5090909094207372,
  16: 0.505138340297895,
  17: 0.5122529647567056,
  18: 0.4940711466220057,
  19: 0.4988142296259582,
  20: 0.499604743412832,
  21: 0.49802371574484783,
  22: 0.505928854131887,
  23: 0.5090909091615865,
  24: 0.4940711462686184,
  25: 0.49249011860063424,
  26: 0.49644268777059475,
  27: 0.49090909123891896,
  28: 0.5035573125827925,
  29: 0.5019762846556577,
  30: 0.4988142296259582,
  31: 0.48774703569091826,
  32: 0.49723320195797405,
  33: 0.507509881446484,
  34: 0.49802371543857893,
  35: 0.5098814232547293},
 {5: 0.488537549784061,
  6: 0.4885375494306738,
  7: 0.4940711462686184,
  8: 0.49249011860063424,
  9: 0.49169960507291105,
  10: 0.4893280636180531,
  11: 0.49802371543857893,
  12: 0.49407

In [13]:
traces = flair_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded Flair dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [10]:
gpt = data.get_gpt()

In [11]:
%%time

concatenated_gpt = {
    fold: [np.concatenate(np.array(statement)) for statement in gpt[fold]['statement']]
    for fold in gpt.keys()
}

gpt_rounds = [calculate_round(concatenated_gpt) for round in range(5)]

Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 52s 5ms/step - loss: 1.1045 - acc: 0.4055 - val_loss: 1.0437 - val_acc: 0.4494
Epoch 2/5
10235/10235 [==============================] - 8s 747us/step - loss: 1.0744 - acc: 0.4264 - val_loss: 1.0432 - val_acc: 0.4626
Epoch 3/5
10235/10235 [==============================] - 8s 736us/step - loss: 1.0680 - acc: 0.4327 - val_loss: 1.0300 - val_acc: 0.4907
Epoch 4/5
10235/10235 [==============================] - 8s 751us/step - loss: 1.0632 - acc: 0.4293 - val_loss: 1.0429 - val_acc: 0.4751
Epoch 5/5
1265/1265 [==============================] - 0s 333us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.1024 - acc: 0.4074 - val_loss: 1.0434 - val_acc: 0.4509
Epoch 2/5
10235/10235 [==============================] - 8s 812us/step - loss: 1.0755 - acc: 0.4197 - val_loss: 1.0297 - val_acc: 0.4821
Epoch 3/5
102

Epoch 3/5
10235/10235 [==============================] - 14s 1ms/step - loss: 1.0625 - acc: 0.4471 - val_loss: 1.0280 - val_acc: 0.5047
Epoch 4/5
10235/10235 [==============================] - 14s 1ms/step - loss: 1.0551 - acc: 0.4475 - val_loss: 1.0202 - val_acc: 0.5062
Epoch 5/5
1265/1265 [==============================] - 1s 644us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 52s 5ms/step - loss: 1.0975 - acc: 0.4228 - val_loss: 1.0316 - val_acc: 0.4805
Epoch 2/5
10235/10235 [==============================] - 15s 1ms/step - loss: 1.0672 - acc: 0.4352 - val_loss: 1.0269 - val_acc: 0.4953
Epoch 3/5
10235/10235 [==============================] - 15s 1ms/step - loss: 1.0612 - acc: 0.4452 - val_loss: 1.0370 - val_acc: 0.4836
Epoch 4/5
10235/10235 [==============================] - 15s 1ms/step - loss: 1.0536 - acc: 0.4552 - val_loss: 1.0266 - val_acc: 0.4961
Epoch 5/5
1265/1265 [==============================] - 1s 670us/st

Epoch 2/5
10235/10235 [==============================] - 8s 824us/step - loss: 1.0726 - acc: 0.4211 - val_loss: 1.0380 - val_acc: 0.4727
Epoch 3/5
10235/10235 [==============================] - 8s 829us/step - loss: 1.0675 - acc: 0.4277 - val_loss: 1.0458 - val_acc: 0.4626
Epoch 4/5
10235/10235 [==============================] - 8s 830us/step - loss: 1.0635 - acc: 0.4319 - val_loss: 1.0320 - val_acc: 0.4829
Epoch 5/5
1265/1265 [==============================] - 0s 363us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 52s 5ms/step - loss: 1.1211 - acc: 0.3879 - val_loss: 1.0425 - val_acc: 0.4603
Epoch 2/5
10235/10235 [==============================] - 9s 887us/step - loss: 1.0813 - acc: 0.4159 - val_loss: 1.0401 - val_acc: 0.4665
Epoch 3/5
10235/10235 [==============================] - 9s 880us/step - loss: 1.0716 - acc: 0.4210 - val_loss: 1.0371 - val_acc: 0.4953
Epoch 4/5
10235/10235 [==============================] - 9s 9

1265/1265 [==============================] - 3s 3ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 94s 9ms/step - loss: 1.0960 - acc: 0.4077 - val_loss: 1.0392 - val_acc: 0.4883
Epoch 2/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0718 - acc: 0.4317 - val_loss: 1.0313 - val_acc: 0.4860
Epoch 3/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0599 - acc: 0.4425 - val_loss: 1.0305 - val_acc: 0.4844
Epoch 4/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0579 - acc: 0.4416 - val_loss: 1.0273 - val_acc: 0.4899
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 92s 9ms/step - loss: 1.0873 - acc: 0.4198 - val_loss: 1.0338 - val_acc: 0.4813
Epoch 2/5
10235/10235 [==============================] - 22s 2ms/step - loss: 1.0695 - acc: 0.4308

10235/10235 [==============================] - 16s 2ms/step - loss: 1.0541 - acc: 0.4537 - val_loss: 1.0224 - val_acc: 0.5062
Epoch 5/5
1265/1265 [==============================] - 1s 738us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 71s 7ms/step - loss: 1.0999 - acc: 0.4157 - val_loss: 1.0376 - val_acc: 0.4782
Epoch 2/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0732 - acc: 0.4295 - val_loss: 1.0327 - val_acc: 0.4875
Epoch 3/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0616 - acc: 0.4444 - val_loss: 1.0262 - val_acc: 0.4891
Epoch 4/5
10235/10235 [==============================] - 16s 2ms/step - loss: 1.0550 - acc: 0.4495 - val_loss: 1.0233 - val_acc: 0.4938
Epoch 5/5
1265/1265 [==============================] - 1s 766us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 76s 7ms/step - loss: 1.1020 - acc: 0.

10235/10235 [==============================] - 99s 10ms/step - loss: 1.0964 - acc: 0.4087 - val_loss: 1.0370 - val_acc: 0.4805
Epoch 2/5
10235/10235 [==============================] - 23s 2ms/step - loss: 1.0724 - acc: 0.4318 - val_loss: 1.0298 - val_acc: 0.4821
Epoch 3/5
10235/10235 [==============================] - 23s 2ms/step - loss: 1.0588 - acc: 0.4414 - val_loss: 1.0266 - val_acc: 0.4875
Epoch 4/5
10235/10235 [==============================] - 23s 2ms/step - loss: 1.0560 - acc: 0.4476 - val_loss: 1.0226 - val_acc: 0.4883
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 104s 10ms/step - loss: 1.0954 - acc: 0.4190 - val_loss: 1.0303 - val_acc: 0.4813
Epoch 2/5
10235/10235 [==============================] - 24s 2ms/step - loss: 1.0671 - acc: 0.4319 - val_loss: 1.0346 - val_acc: 0.4930
Epoch 3/5
10235/10235 [==============================] - 24s 2ms/step - los

1265/1265 [==============================] - 1s 790us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 85s 8ms/step - loss: 1.1138 - acc: 0.4164 - val_loss: 1.0342 - val_acc: 0.5062
Epoch 2/5
10235/10235 [==============================] - 17s 2ms/step - loss: 1.0725 - acc: 0.4407 - val_loss: 1.0334 - val_acc: 0.4891
Epoch 3/5
10235/10235 [==============================] - 17s 2ms/step - loss: 1.0601 - acc: 0.4431 - val_loss: 1.0246 - val_acc: 0.5047
Epoch 4/5
10235/10235 [==============================] - 17s 2ms/step - loss: 1.0565 - acc: 0.4538 - val_loss: 1.0299 - val_acc: 0.4891
Epoch 5/5
1265/1265 [==============================] - 1s 1ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 95s 9ms/step - loss: 1.1084 - acc: 0.4075 - val_loss: 1.0284 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 18s 2ms/step - loss: 1.0705 - acc: 0.43

10235/10235 [==============================] - 9s 924us/step - loss: 1.0713 - acc: 0.4249 - val_loss: 1.0328 - val_acc: 0.4914
Epoch 4/5
10235/10235 [==============================] - 9s 927us/step - loss: 1.0651 - acc: 0.4289 - val_loss: 1.0331 - val_acc: 0.4875
Epoch 5/5
1265/1265 [==============================] - 1s 428us/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 79s 8ms/step - loss: 1.1144 - acc: 0.4007 - val_loss: 1.0396 - val_acc: 0.4829
Epoch 2/5
10235/10235 [==============================] - 11s 1ms/step - loss: 1.0753 - acc: 0.4270 - val_loss: 1.0316 - val_acc: 0.4945
Epoch 3/5
10235/10235 [==============================] - 11s 1ms/step - loss: 1.0671 - acc: 0.4292 - val_loss: 1.0368 - val_acc: 0.4914
Epoch 4/5
10235/10235 [==============================] - 11s 1ms/step - loss: 1.0615 - acc: 0.4368 - val_loss: 1.0321 - val_acc: 0.4891
Epoch 5/5
1265/1265 [==============================] - 1s 486us/step
Train

1265/1265 [==============================] - 3s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0888 - acc: 0.4174 - val_loss: 1.0360 - val_acc: 0.4805
Epoch 2/5
10235/10235 [==============================] - 25s 2ms/step - loss: 1.0662 - acc: 0.4337 - val_loss: 1.0339 - val_acc: 0.4883
Epoch 3/5
10235/10235 [==============================] - 25s 2ms/step - loss: 1.0607 - acc: 0.4330 - val_loss: 1.0309 - val_acc: 0.4945
Epoch 4/5
10235/10235 [==============================] - 25s 2ms/step - loss: 1.0557 - acc: 0.4450 - val_loss: 1.0256 - val_acc: 0.4836
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0915 - acc: 0.4164 - val_loss: 1.0378 - val_acc: 0.4813
Epoch 2/5
10235/10235 [==============================] - 26s 3ms/step - loss: 1.0686 - acc: 0.

In [12]:
gpt_rounds

[{5: 0.4395256920294328,
  6: 0.445849802607133,
  7: 0.47114624534199834,
  8: 0.44743083036935377,
  9: 0.46245059321520354,
  10: 0.46166007938121145,
  11: 0.46640316238516405,
  12: 0.46245059321520354,
  13: 0.44664031658248,
  14: 0.4545454549224009,
  15: 0.4640316208831878,
  16: 0.47826086984792715,
  17: 0.4750988145590771,
  18: 0.474308300725085,
  19: 0.46640316243228236,
  20: 0.4727272730099825,
  21: 0.4727272730099825,
  22: 0.47984189751591133,
  23: 0.4703557315551245,
  24: 0.47351778658482396,
  25: 0.47905138377615586,
  26: 0.4750988145119588,
  27: 0.46798419005314823,
  28: 0.46877470393425863,
  29: 0.474308300725085,
  30: 0.48142292518389557,
  31: 0.4561264825432668,
  32: 0.47826086984792715,
  33: 0.46640316238516405,
  34: 0.4632411070020774,
  35: 0.474308300725085},
 {5: 0.43636363669346445,
  6: 0.44901185799021964,
  7: 0.4584980239981248,
  8: 0.44664031648824337,
  9: 0.46245059321520354,
  10: 0.46245059321520354,
  11: 0.46482213471717987,
  12:

In [14]:
traces = gpt_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'Test set accuracy of padded GPT dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [19]:
x = list(range(5, 36))
x_rev = x[::-1]

# BERT
bert_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in bert_rounds]))
bert_y = [np.average(row) for row in bert_matrix]
bert_y_upper = [row.max() for row in bert_matrix]
bert_y_lower = [row.min() for row in bert_matrix]
bert_y_lower = bert_y_lower[::-1]

bert1 = go.Scatter(
    x = x + x_rev,
    y = bert_y_upper + bert_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,100,80,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'BERT',
)
bert2 = go.Scatter(
    x = x,
    y = bert_y,
    line = dict(color='rgb(0,100,80)'),
    mode = 'lines+markers',
    name = 'BERT',
)

# ELMo
elmo_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in elmo_rounds]))
elmo_y = [np.average(row) for row in elmo_matrix]
elmo_y_upper = [row.max() for row in elmo_matrix]
elmo_y_lower = [row.min() for row in elmo_matrix]
elmo_y_lower = elmo_y_lower[::-1]

elmo1 = go.Scatter(
    x = x + x_rev,
    y = elmo_y_upper + elmo_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,176,246,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'ELMo',
)
elmo2 = go.Scatter(
    x = x,
    y = elmo_y,
    line = dict(color='rgb(0,176,246)'),
    mode = 'lines+markers',
    name = 'ELMo',
)

# Transformer-XL
transformerxl_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in transformerxl_rounds]))
transformerxl_y = [np.average(row) for row in transformerxl_matrix]
transformerxl_y_upper = [row.max() for row in transformerxl_matrix]
transformerxl_y_lower = [row.min() for row in transformerxl_matrix]
transformerxl_y_lower = transformerxl_y_lower[::-1]

transformerxl1 = go.Scatter(
    x = x + x_rev,
    y = transformerxl_y_upper + transformerxl_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(231,107,243,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'Transformer-XL',
)
transformerxl2 = go.Scatter(
    x = x,
    y = transformerxl_y,
    line = dict(color='rgb(231,107,243)'),
    mode = 'lines+markers',
    name = 'Transformer-XL',
)

# Flair
flair_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in flair_rounds]))
flair_y = [np.average(row) for row in flair_matrix]
flair_y_upper = [row.max() for row in flair_matrix]
flair_y_lower = [row.min() for row in flair_matrix]
flair_y_lower = flair_y_lower[::-1]

flair1 = go.Scatter(
    x = x + x_rev,
    y = flair_y_upper + flair_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(205,12,24,0.2)',
    line = dict(color = 'rgba(205,12,24,0)'),
    showlegend = False,
    name = 'Flair',
)
flair2 = go.Scatter(
    x = x,
    y = flair_y,
    line = dict(color='rgb(205,12,24)'),
    mode = 'lines+markers',
    name = 'Flair',
)

# GPT
gpt_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in gpt_rounds]))
gpt_y = [np.average(row) for row in gpt_matrix]
gpt_y_upper = [row.max() for row in gpt_matrix]
gpt_y_lower = [row.min() for row in gpt_matrix]
gpt_y_lower = gpt_y_lower[::-1]

gpt1 = go.Scatter(
    x = x + x_rev,
    y = gpt_y_upper + gpt_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(81, 45, 168, 0.2)',
    line = dict(color = 'rgba(81, 45, 168, 0)'),
    showlegend = False,
    name = 'GPT',
)
gpt2 = go.Scatter(
    x = x,
    y = gpt_y,
    line = dict(color='rgb(81, 45, 168)'),
    mode = 'lines+markers',
    name = 'GPT',
)

# Baseline and highest performance
baseline = go.Scatter(
    x = x,
    y = [0.4428 for i in x],
    line = dict(color='rgb(0,0,0)'),
    mode = 'lines',
    name = 'Majority vote',
)
best_past_performance = go.Scatter(
    x = x,
    y = [0.4903 for i in x],
    mode = 'lines',
    name = 'Khurana (2017)',
)


data = [bert1, bert2, elmo1, elmo2, transformerxl1, transformerxl2, flair1, flair2, gpt1, gpt2, baseline, best_past_performance]
layout = go.Layout(
    title = 'Bi-LSTM test set accuracy of padded datasets with variable maximum lengths',
    paper_bgcolor = 'rgb(255,255,255)',
    plot_bgcolor = 'rgb(229,229,229)',
    xaxis = dict(
        gridcolor = 'rgb(255,255,255)',
        range = [5,35],
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
)

fig = go.Figure(data = data, layout = layout)

iplot(fig)

<hr>

## Convolutional neural networks

In [6]:
def get_cnn_score(X_train, X_test, X_validation, y_train = general['train']['label'], y_test = general['test']['label'], y_validation = general['validation']['label'], reshape = True):
    # Rearrange data types    
    params = locals().copy()    
    inputs = {
        dataset: np.array(params[dataset])
        for dataset in params.keys()
    }
    
    # Reshape datasets
    for dataset in inputs.keys():
        if dataset[0:1] == 'X':
            if reshape:
                inputs[dataset] = np.reshape(inputs[dataset], (inputs[dataset].shape[0], inputs[dataset].shape[1], 1))
            
        elif dataset[0:1] == 'y':
            inputs[dataset] = np_utils.to_categorical(np.array(inputs[dataset]), 3)
            
    # Set model parameters
    epochs = 5
    batch_size = 32
    input_shape =  inputs['X_train'].shape
    
    # Create the model
    model = Sequential()
    model.add(Conv1D(128, kernel_size = 2, activation='relu', input_shape = (input_shape[1], input_shape[2]), data_format = 'channels_first'))
    model.add(Conv1D(128, kernel_size = 3, activation='relu'))
    model.add(Conv1D(128, kernel_size = 4, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Flatten())
    model.add(Dense(3, activation = 'softmax'))
    model.compile('sgd', 'categorical_crossentropy', metrics = ['accuracy']) 
    
    # Fit the training set over the model and correct on the validation set
    model.fit(inputs['X_train'], inputs['y_train'],
            batch_size = batch_size,
            epochs = epochs,
            validation_data = (inputs['X_validation'], inputs['y_validation']))
    
    # Get score over the test set
    score, acc = model.evaluate(inputs['X_test'], inputs['y_test'])
    print(acc)
    return acc

In [5]:
concatenated_bert = {
    dataset: [np.concatenate(np.array(statement)) for statement in bert[dataset].statement]
    for dataset in bert.keys()
}

In [7]:
def calculate_round(dataset):
    # Store accuracies
    accuracies = {
        padding_len: 0.0 for padding_len in list(range(5,36))
    }

    for max_len in accuracies.keys():
        padded_dataset = {
            fold: sequence.pad_sequences(dataset[fold], maxlen = max_len, dtype = float)
            for fold in dataset.keys()
        }

        accuracies[max_len] = get_cnn_score(padded_dataset['train'], padded_dataset['test'], padded_dataset['validation'], reshape = False)
        
    print(accuracies)
    return accuracies

In [10]:
%%time

cnn_bert_rounds = [calculate_round(concatenated_bert) for round in range(5)]

Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 116s 11ms/step - loss: 1.0628 - acc: 0.4374 - val_loss: 1.0260 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 115s 11ms/step - loss: 1.0405 - acc: 0.4690 - val_loss: 1.0127 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 112s 11ms/step - loss: 1.0328 - acc: 0.4813 - val_loss: 1.0137 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 114s 11ms/step - loss: 1.0260 - acc: 0.4858 - val_loss: 1.0025 - val_acc: 0.5156
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5067193679658791
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 112s 11ms/step - loss: 1.0658 - acc: 0.4321 - val_loss: 1.0336 - val_acc: 0.4766
Epoch 2/5
10235/10235 [==============================] - 115s 11ms/step - loss: 1.0416 - acc: 0.4671 - val_loss: 1.0154 - val_

10235/10235 [==============================] - 133s 13ms/step - loss: 1.0735 - acc: 0.4336 - val_loss: 1.0215 - val_acc: 0.5000
Epoch 2/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0370 - acc: 0.4797 - val_loss: 1.0095 - val_acc: 0.5031
Epoch 3/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0269 - acc: 0.4952 - val_loss: 1.0007 - val_acc: 0.5148
Epoch 4/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0114 - acc: 0.5042 - val_loss: 1.0095 - val_acc: 0.5132
Epoch 5/5
1265/1265 [==============================] - 6s 5ms/step
0.507509881752753
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 138s 13ms/step - loss: 1.0652 - acc: 0.4345 - val_loss: 1.0294 - val_acc: 0.5055
Epoch 2/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0373 - acc: 0.4734 - val_loss: 1.0120 - val_acc: 0.5093
Epoch 3/5
10235/10235 [=========================

10235/10235 [==============================] - 124s 12ms/step - loss: 1.0096 - acc: 0.5057 - val_loss: 0.9964 - val_acc: 0.5187
Epoch 5/5
1265/1265 [==============================] - 6s 4ms/step
0.4980237156977295
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 139s 14ms/step - loss: 1.0707 - acc: 0.4367 - val_loss: 1.0412 - val_acc: 0.4548
Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0401 - acc: 0.4781 - val_loss: 1.0096 - val_acc: 0.5101
Epoch 3/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0221 - acc: 0.4986 - val_loss: 1.0191 - val_acc: 0.5047
Epoch 4/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0128 - acc: 0.5006 - val_loss: 0.9964 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.4814229252310138
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [===========================

10235/10235 [==============================] - 141s 14ms/step - loss: 1.0711 - acc: 0.4366 - val_loss: 1.0196 - val_acc: 0.5117
Epoch 2/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0364 - acc: 0.4766 - val_loss: 1.0049 - val_acc: 0.5109
Epoch 3/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0213 - acc: 0.4958 - val_loss: 1.0057 - val_acc: 0.5187
Epoch 4/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0111 - acc: 0.5043 - val_loss: 0.9927 - val_acc: 0.5218
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.49723320191085574
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 147s 14ms/step - loss: 1.0712 - acc: 0.4286 - val_loss: 1.0225 - val_acc: 0.4969
Epoch 2/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0429 - acc: 0.4731 - val_loss: 1.0137 - val_acc: 0.4992
Epoch 3/5
10235/10235 [=======================

10235/10235 [==============================] - 126s 12ms/step - loss: 1.0120 - acc: 0.5033 - val_loss: 0.9987 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.5106719370416031
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 152s 15ms/step - loss: 1.0685 - acc: 0.4330 - val_loss: 1.0268 - val_acc: 0.4821
Epoch 2/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0380 - acc: 0.4801 - val_loss: 1.0182 - val_acc: 0.5164
Epoch 3/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0225 - acc: 0.4940 - val_loss: 1.0065 - val_acc: 0.5062
Epoch 4/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0065 - acc: 0.5085 - val_loss: 1.0135 - val_acc: 0.5109
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.5059288540847687
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [===========================

10235/10235 [==============================] - 163s 16ms/step - loss: 1.0770 - acc: 0.4317 - val_loss: 1.0303 - val_acc: 0.4961
Epoch 2/5
10235/10235 [==============================] - 141s 14ms/step - loss: 1.0416 - acc: 0.4709 - val_loss: 1.0174 - val_acc: 0.4945
Epoch 3/5
10235/10235 [==============================] - 137s 13ms/step - loss: 1.0239 - acc: 0.4954 - val_loss: 1.0038 - val_acc: 0.5078
Epoch 4/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0123 - acc: 0.5000 - val_loss: 1.0056 - val_acc: 0.5179
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.5122529648038239
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 167s 16ms/step - loss: 1.0735 - acc: 0.4315 - val_loss: 1.0252 - val_acc: 0.5093
Epoch 2/5
10235/10235 [==============================] - 140s 14ms/step - loss: 1.0392 - acc: 0.4754 - val_loss: 1.0253 - val_acc: 0.5023
Epoch 3/5
10235/10235 [========================

In [20]:
cnn_bert_rounds

[{5: 0.5067193679658791,
  6: 0.5162055339737843,
  7: 0.5130434782844288,
  8: 0.5225296443865705,
  9: 0.5098814232547293,
  10: 0.505928854131887,
  11: 0.5027667987016821,
  12: 0.507509881752753,
  13: 0.49644268807686365,
  14: 0.48458498056698224,
  15: 0.507509881752753,
  16: 0.4996047431536814,
  17: 0.5114624508755952,
  18: 0.49802371574484783,
  19: 0.5011857710808162,
  20: 0.5043478261105157,
  21: 0.5209486169777369,
  22: 0.516205533926666,
  23: 0.49723320191085574,
  24: 0.4758893284401875,
  25: 0.5098814233018476,
  26: 0.5043478264639029,
  27: 0.5035573126299108,
  28: 0.4877470355966817,
  29: 0.5114624509698318,
  30: 0.5083003953275944,
  31: 0.5209486169306186,
  32: 0.5090909094678555,
  33: 0.5035573125827925,
  34: 0.5169960478077764,
  35: 0.5035573125827925},
 {5: 0.5122529644975549,
  6: 0.5027667984896498,
  7: 0.485375494188942,
  8: 0.5177865616417685,
  9: 0.5106719371358397,
  10: 0.5138339924246897,
  11: 0.48695652180980786,
  12: 0.4861660080229

In [21]:
traces = cnn_bert_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded BERT dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [7]:
concatenated_elmo = {
    dataset: [np.concatenate(np.array(statement)) for statement in elmo[dataset].statement]
    for dataset in elmo.keys()
}

In [8]:
%%time

cnn_elmo_rounds = [calculate_round(concatenated_elmo) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-06 17:27:55,650 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-06 17:27:55,736 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-06 17:27:55,831 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

1265/1265 [==============================] - ETA:  - 5s 4ms/step
0.5185770751223734
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0726 - acc: 0.4361 - val_loss: 1.0170 - val_acc: 0.5008
Epoch 2/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0353 - acc: 0.4823 - val_loss: 1.0163 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0192 - acc: 0.4979 - val_loss: 1.0048 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 122s 12ms/step - loss: 0.9997 - acc: 0.5165 - val_loss: 1.0008 - val_acc: 0.5140
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.507509881752753
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0737 - acc: 0.4322 - val_loss: 1.0191 - val_acc: 0.5008
Epoch 2/5
10235/10235 [====================

1265/1265 [==============================] - 9s 7ms/step
0.513833992471808
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 167s 16ms/step - loss: 1.0716 - acc: 0.4357 - val_loss: 1.0388 - val_acc: 0.4899
Epoch 2/5
10235/10235 [==============================] - 139s 14ms/step - loss: 1.0345 - acc: 0.4812 - val_loss: 1.0053 - val_acc: 0.5101
Epoch 3/5
10235/10235 [==============================] - 140s 14ms/step - loss: 1.0126 - acc: 0.5059 - val_loss: 1.0064 - val_acc: 0.5109
Epoch 4/5
10235/10235 [==============================] - 139s 14ms/step - loss: 0.9965 - acc: 0.5195 - val_loss: 0.9943 - val_acc: 0.5234
Epoch 5/5
1265/1265 [==============================] - 8s 7ms/step
0.5003952569405552
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 173s 17ms/step - loss: 1.0696 - acc: 0.4360 - val_loss: 1.0258 - val_acc: 0.5000
Epoch 2/5
10235/10235 [============================

1265/1265 [==============================] - 5s 4ms/step
0.5130434782844288
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0715 - acc: 0.4344 - val_loss: 1.0310 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 122s 12ms/step - loss: 1.0360 - acc: 0.4796 - val_loss: 1.0304 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0128 - acc: 0.4983 - val_loss: 1.0062 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 123s 12ms/step - loss: 1.0000 - acc: 0.5127 - val_loss: 1.0049 - val_acc: 0.5234
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5098814233018476
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 128s 13ms/step - loss: 1.0702 - acc: 0.4283 - val_loss: 1.0165 - val_acc: 0.5008
Epoch 2/5
10235/10235 [===========================

1265/1265 [==============================] - 9s 7ms/step
0.5217391307646106
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 164s 16ms/step - loss: 1.0778 - acc: 0.4259 - val_loss: 1.0365 - val_acc: 0.4977
Epoch 2/5
10235/10235 [==============================] - 132s 13ms/step - loss: 1.0353 - acc: 0.4819 - val_loss: 1.0078 - val_acc: 0.5335
Epoch 3/5
10235/10235 [==============================] - 132s 13ms/step - loss: 1.0154 - acc: 0.5017 - val_loss: 0.9910 - val_acc: 0.5249
Epoch 4/5
10235/10235 [==============================] - 133s 13ms/step - loss: 0.9936 - acc: 0.5215 - val_loss: 0.9856 - val_acc: 0.5304
Epoch 5/5
1265/1265 [==============================] - 8s 6ms/step
0.5083003955867451
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 169s 16ms/step - loss: 1.0694 - acc: 0.4379 - val_loss: 1.0315 - val_acc: 0.4961
Epoch 2/5
10235/10235 [===========================

...

10235/10235 [==============================] - 163s 16ms/step - loss: 1.0771 - acc: 0.4252 - val_loss: 1.0210 - val_acc: 0.5031
Epoch 2/5
10235/10235 [==============================] - 131s 13ms/step - loss: 1.0373 - acc: 0.4782 - val_loss: 1.0115 - val_acc: 0.5187
Epoch 3/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0168 - acc: 0.5003 - val_loss: 0.9971 - val_acc: 0.5312
Epoch 4/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0000 - acc: 0.5188 - val_loss: 0.9941 - val_acc: 0.5241
Epoch 5/5
1265/1265 [==============================] - 8s 6ms/step
0.5114624509227135
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 162s 16ms/step - loss: 1.0850 - acc: 0.4295 - val_loss: 1.0199 - val_acc: 0.5070
Epoch 2/5
10235/10235 [==============================] - 129s 13ms/step - loss: 1.0409 - acc: 0.4747 - val_loss: 1.0107 - val_acc: 0.5055
Epoch 3/5
10235/10235 [========================

Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0411 - acc: 0.4689 - val_loss: 1.0262 - val_acc: 0.5000
Epoch 3/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0268 - acc: 0.4912 - val_loss: 1.0182 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0152 - acc: 0.5000 - val_loss: 1.0038 - val_acc: 0.5125
Epoch 5/5
1265/1265 [==============================] - 5s 4ms/step
0.5098814232547293
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0691 - acc: 0.4301 - val_loss: 1.0168 - val_acc: 0.5008
Epoch 2/5
10235/10235 [==============================] - 120s 12ms/step - loss: 1.0389 - acc: 0.4739 - val_loss: 1.0149 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 144s 14ms/step - loss: 1.0221 - acc: 0.4914 - val_loss: 0.9999 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============

10235/10235 [==============================] - 131s 13ms/step - loss: 1.0389 - acc: 0.4823 - val_loss: 1.0070 - val_acc: 0.5156
Epoch 3/5
10235/10235 [==============================] - 127s 12ms/step - loss: 1.0159 - acc: 0.5061 - val_loss: 0.9986 - val_acc: 0.5132
Epoch 4/5
10235/10235 [==============================] - 127s 12ms/step - loss: 0.9977 - acc: 0.5173 - val_loss: 0.9886 - val_acc: 0.5202
Epoch 5/5
1265/1265 [==============================] - 8s 7ms/step
0.5083003956338633
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 163s 16ms/step - loss: 1.0747 - acc: 0.4308 - val_loss: 1.0229 - val_acc: 0.4938
Epoch 2/5
10235/10235 [==============================] - 128s 13ms/step - loss: 1.0381 - acc: 0.4708 - val_loss: 1.0035 - val_acc: 0.5234
Epoch 3/5
10235/10235 [==============================] - 128s 13ms/step - loss: 1.0169 - acc: 0.5021 - val_loss: 0.9934 - val_acc: 0.5195
Epoch 4/5
10235/10235 [========================

Epoch 3/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0300 - acc: 0.4841 - val_loss: 1.0144 - val_acc: 0.5171
Epoch 4/5
10235/10235 [==============================] - 126s 12ms/step - loss: 1.0172 - acc: 0.5008 - val_loss: 1.0105 - val_acc: 0.5125
Epoch 5/5
1265/1265 [==============================] - 6s 5ms/step
0.507509881446484
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 124s 12ms/step - loss: 1.0650 - acc: 0.4336 - val_loss: 1.0224 - val_acc: 0.4969
Epoch 2/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0398 - acc: 0.4695 - val_loss: 1.0067 - val_acc: 0.5055
Epoch 3/5
10235/10235 [==============================] - 118s 12ms/step - loss: 1.0227 - acc: 0.4938 - val_loss: 1.0032 - val_acc: 0.5117
Epoch 4/5
10235/10235 [==============================] - 118s 11ms/step - loss: 1.0130 - acc: 0.5009 - val_loss: 0.9992 - val_acc: 0.5125
Epoch 5/5
1265/1265 [=================

10235/10235 [==============================] - 130s 13ms/step - loss: 1.0178 - acc: 0.5014 - val_loss: 0.9965 - val_acc: 0.5249
Epoch 4/5
10235/10235 [==============================] - 130s 13ms/step - loss: 1.0000 - acc: 0.5150 - val_loss: 0.9935 - val_acc: 0.5202
Epoch 5/5
1265/1265 [==============================] - 8s 7ms/step
0.5138339924246897
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 165s 16ms/step - loss: 1.0795 - acc: 0.4297 - val_loss: 1.0287 - val_acc: 0.4922
Epoch 2/5
10235/10235 [==============================] - 130s 13ms/step - loss: 1.0383 - acc: 0.4809 - val_loss: 1.0086 - val_acc: 0.5156
Epoch 3/5
10235/10235 [==============================] - 125s 12ms/step - loss: 1.0171 - acc: 0.5005 - val_loss: 0.9995 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 126s 12ms/step - loss: 0.9985 - acc: 0.5206 - val_loss: 0.9990 - val_acc: 0.5148
Epoch 5/5
1265/1265 [==========================

In [9]:
cnn_elmo_rounds

[{5: 0.49565217424287156,
  6: 0.5114624506164446,
  7: 0.5169960477606581,
  8: 0.5122529644504367,
  9: 0.49249011860063424,
  10: 0.49644268807686365,
  11: 0.5075098817998712,
  12: 0.5185770751223734,
  13: 0.507509881752753,
  14: 0.4988142295788399,
  15: 0.5043478264639029,
  16: 0.5122529647567056,
  17: 0.5067193679187609,
  18: 0.5090909094207372,
  19: 0.513833992471808,
  20: 0.5059288540847687,
  21: 0.5209486166243497,
  22: 0.49565217424287156,
  23: 0.5122529647567056,
  24: 0.5098814229484603,
  25: 0.5272727276025554,
  26: 0.5225296445514844,
  27: 0.5146245063058001,
  28: 0.5051383402507766,
  29: 0.5059288537784998,
  30: 0.5169960477606581,
  31: 0.5051383402507766,
  32: 0.513833992471808,
  33: 0.5003952569405552,
  34: 0.5106719370887214,
  35: 0.5019762849148083},
 {5: 0.5043478261105157,
  6: 0.5114624509698318,
  7: 0.5304347829385238,
  8: 0.5067193679658791,
  9: 0.499604743412832,
  10: 0.5130434782844288,
  11: 0.5098814233018476,
  12: 0.5051383402978

In [10]:
traces = cnn_elmo_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded ELMo dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [5]:
transformerxl = data.get_transformerxl()

In [6]:
concatenated_transformerxl = {
    dataset: [np.concatenate(np.array(statement)) for statement in transformerxl[dataset].statement]
    for dataset in transformerxl.keys()
}

In [9]:
%%time

cnn_transformerxl_rounds = [calculate_round(concatenated_transformerxl) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-10 15:58:40,328 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-10 15:58:40,431 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-10 15:58:40,520 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

1265/1265 [==============================] - 2s 1ms/step
0.49090909093265006
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0632 - acc: 0.4306 - val_loss: 1.0548 - val_acc: 0.3863
Epoch 2/5
10235/10235 [==============================] - 41s 4ms/step - loss: 1.0544 - acc: 0.4400 - val_loss: 1.0278 - val_acc: 0.4836
Epoch 3/5
10235/10235 [==============================] - 41s 4ms/step - loss: 1.0473 - acc: 0.4571 - val_loss: 1.0240 - val_acc: 0.5047
Epoch 4/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.0413 - acc: 0.4692 - val_loss: 1.0219 - val_acc: 0.4953
Epoch 5/5
1265/1265 [==============================] - 2s 1ms/step
0.4885375497369427
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0595 - acc: 0.4378 - val_loss: 1.0378 - val_acc: 0.4992
Epoch 2/5
10235/10235 [==============================] - 42

10235/10235 [==============================] - 44s 4ms/step - loss: 1.0504 - acc: 0.4485 - val_loss: 1.0276 - val_acc: 0.4821
Epoch 3/5
10235/10235 [==============================] - 44s 4ms/step - loss: 1.0426 - acc: 0.4667 - val_loss: 1.0220 - val_acc: 0.4930
Epoch 4/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.0370 - acc: 0.4779 - val_loss: 1.0207 - val_acc: 0.5039
Epoch 5/5
1265/1265 [==============================] - 2s 1ms/step
0.49169960507291105
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 41s 4ms/step - loss: 1.0590 - acc: 0.4353 - val_loss: 1.0377 - val_acc: 0.4945
Epoch 2/5
10235/10235 [==============================] - 40s 4ms/step - loss: 1.0528 - acc: 0.4404 - val_loss: 1.0294 - val_acc: 0.5078
Epoch 3/5
10235/10235 [==============================] - 40s 4ms/step - loss: 1.0441 - acc: 0.4609 - val_loss: 1.0186 - val_acc: 0.5070
Epoch 4/5
10235/10235 [==============================] - 3

10235/10235 [==============================] - 44s 4ms/step - loss: 1.0404 - acc: 0.4687 - val_loss: 1.0067 - val_acc: 0.5101
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
0.47905138372903755
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 45s 4ms/step - loss: 1.0625 - acc: 0.4347 - val_loss: 1.0411 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.0509 - acc: 0.4515 - val_loss: 1.0282 - val_acc: 0.4813
Epoch 3/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0426 - acc: 0.4650 - val_loss: 1.0288 - val_acc: 0.5093
Epoch 4/5
10235/10235 [==============================] - 63s 6ms/step - loss: 1.0346 - acc: 0.4733 - val_loss: 1.0148 - val_acc: 0.5070
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
0.48063241139702173
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 5

1265/1265 [==============================] - 2s 1ms/step
0.48379446673299015
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 47s 5ms/step - loss: 1.0633 - acc: 0.4227 - val_loss: 1.0352 - val_acc: 0.4790
Epoch 2/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0607 - acc: 0.4407 - val_loss: 1.0386 - val_acc: 0.5039
Epoch 3/5
10235/10235 [==============================] - 43s 4ms/step - loss: 1.0751 - acc: 0.4381 - val_loss: 1.0251 - val_acc: 0.4836
Epoch 4/5
10235/10235 [==============================] - 43s 4ms/step - loss: 3.3349 - acc: 0.4073 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 5/5
1265/1265 [==============================] - 2s 1ms/step
0.3541501976402381
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 47s 5ms/step - loss: 1.0616 - acc: 0.4305 - val_loss: 1.0331 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 4

10235/10235 [==============================] - 45s 4ms/step - loss: 1.0530 - acc: 0.4407 - val_loss: 1.0313 - val_acc: 0.4798
Epoch 3/5
10235/10235 [==============================] - 45s 4ms/step - loss: 1.0487 - acc: 0.4549 - val_loss: 1.0225 - val_acc: 0.4798
Epoch 4/5
10235/10235 [==============================] - 45s 4ms/step - loss: 1.0409 - acc: 0.4691 - val_loss: 1.0173 - val_acc: 0.5132
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
0.5043478264167846
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 49s 5ms/step - loss: 1.0585 - acc: 0.4338 - val_loss: 1.0325 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 45s 4ms/step - loss: 1.0514 - acc: 0.4467 - val_loss: 1.0257 - val_acc: 0.4883
Epoch 3/5
10235/10235 [==============================] - 45s 4ms/step - loss: 1.0437 - acc: 0.4620 - val_loss: 1.0314 - val_acc: 0.4891
Epoch 4/5
10235/10235 [==============================] - 47

10235/10235 [==============================] - 44s 4ms/step - loss: 1.0367 - acc: 0.4726 - val_loss: 1.0085 - val_acc: 0.5047
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
0.4861660082349664
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 48s 5ms/step - loss: 1.0624 - acc: 0.4326 - val_loss: 1.0396 - val_acc: 0.4961
Epoch 2/5
10235/10235 [==============================] - 44s 4ms/step - loss: 1.0541 - acc: 0.4444 - val_loss: 1.0301 - val_acc: 0.5117
Epoch 3/5
10235/10235 [==============================] - 44s 4ms/step - loss: 1.0441 - acc: 0.4617 - val_loss: 1.0166 - val_acc: 0.4945
Epoch 4/5
10235/10235 [==============================] - 44s 4ms/step - loss: 1.0353 - acc: 0.4749 - val_loss: 1.0271 - val_acc: 0.5031
Epoch 5/5
1265/1265 [==============================] - 2s 1ms/step
0.49090909123891896
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 51

1265/1265 [==============================] - 2s 1ms/step
0.49723320191085574
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 48s 5ms/step - loss: 1.0619 - acc: 0.4305 - val_loss: 1.0346 - val_acc: 0.4790
Epoch 2/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.0536 - acc: 0.4462 - val_loss: 1.0285 - val_acc: 0.5023
Epoch 3/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.0444 - acc: 0.4645 - val_loss: 1.0185 - val_acc: 0.5156
Epoch 4/5
10235/10235 [==============================] - 42s 4ms/step - loss: 1.0376 - acc: 0.4745 - val_loss: 1.0077 - val_acc: 0.5226
Epoch 5/5
1265/1265 [==============================] - 2s 1ms/step
0.4893280635709348
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 49s 5ms/step - loss: 1.0621 - acc: 0.4296 - val_loss: 1.0381 - val_acc: 0.4805
Epoch 2/5
10235/10235 [==============================] - 42

In [10]:
cnn_transformerxl_rounds

[{5: 0.4924901189069032,
  6: 0.49644268802974534,
  7: 0.49011857740492687,
  8: 0.49090909123891896,
  9: 0.4869565221160768,
  10: 0.4869565220689585,
  11: 0.4869565220689585,
  12: 0.4885375497369427,
  13: 0.47826086989504546,
  14: 0.49644268812398196,
  15: 0.4893280636180531,
  16: 0.4924901189069032,
  17: 0.4940711465748874,
  18: 0.49169960507291105,
  19: 0.4932806327408953,
  20: 0.48379446673299015,
  21: 0.49169960512002936,
  22: 0.49090909093265006,
  23: 0.4885375497369427,
  24: 0.4735177868910929,
  25: 0.4924901189069032,
  26: 0.49802371574484783,
  27: 0.49723320191085574,
  28: 0.4861660082349664,
  29: 0.49249011895402145,
  30: 0.48458498056698224,
  31: 0.4774703561081717,
  32: 0.48379446673299015,
  33: 0.49090909123891896,
  34: 0.48300395289899806,
  35: 0.4940711466220057},
 {5: 0.474308300725085,
  6: 0.49090909123891896,
  7: 0.49486166040887947,
  8: 0.48063241139702173,
  9: 0.4830039529461163,
  10: 0.4869565221160768,
  11: 0.49169960507291105,
  

In [20]:
traces = cnn_transformerxl_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded Transformer-XL dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [10]:
flair = data.get_flair()

In [11]:
concatenated_flair = {
    dataset: [np.concatenate(np.array(statement)) for statement in flair[dataset].statement]
    for dataset in flair.keys()
}

In [12]:
%%time

cnn_flair_rounds = [calculate_round(concatenated_flair) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-11 19:18:17,751 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-11 19:18:17,911 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-11 19:18:18,420 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

1265/1265 [==============================] - 7s 6ms/step
0.488537549784061
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 176s 17ms/step - loss: 1.0606 - acc: 0.4250 - val_loss: 1.0370 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 166s 16ms/step - loss: 1.0523 - acc: 0.4494 - val_loss: 1.0304 - val_acc: 0.4813
Epoch 3/5
10235/10235 [==============================] - 166s 16ms/step - loss: 1.0458 - acc: 0.4583 - val_loss: 1.0204 - val_acc: 0.4821
Epoch 4/5
10235/10235 [==============================] - 166s 16ms/step - loss: 1.0395 - acc: 0.4711 - val_loss: 1.0186 - val_acc: 0.4969
Epoch 5/5
1265/1265 [==============================] - 9s 7ms/step
0.49090909093265006
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 180s 18ms/step - loss: 1.0595 - acc: 0.4346 - val_loss: 1.0380 - val_acc: 0.4798
Epoch 2/5
10235/10235 [===========================

1265/1265 [==============================] - 12s 10ms/step
0.49723320195797405
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 238s 23ms/step - loss: 1.0619 - acc: 0.4286 - val_loss: 1.0414 - val_acc: 0.4836
Epoch 2/5
10235/10235 [==============================] - 193s 19ms/step - loss: 1.0513 - acc: 0.4429 - val_loss: 1.0297 - val_acc: 0.4914
Epoch 3/5
10235/10235 [==============================] - 190s 19ms/step - loss: 1.0423 - acc: 0.4573 - val_loss: 1.0164 - val_acc: 0.4852
Epoch 4/5
10235/10235 [==============================] - 190s 19ms/step - loss: 1.0362 - acc: 0.4740 - val_loss: 1.0203 - val_acc: 0.5023
Epoch 5/5
1265/1265 [==============================] - 13s 11ms/step
0.5003952571997058
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 238s 23ms/step - loss: 1.0603 - acc: 0.4306 - val_loss: 1.0388 - val_acc: 0.4798
Epoch 2/5
10235/10235 [======================

1265/1265 [==============================] - 7s 6ms/step
0.4877470359500689
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 169s 16ms/step - loss: 1.0567 - acc: 0.4334 - val_loss: 1.0398 - val_acc: 0.4844
Epoch 2/5
10235/10235 [==============================] - 166s 16ms/step - loss: 1.0528 - acc: 0.4475 - val_loss: 1.0303 - val_acc: 0.4868
Epoch 3/5
10235/10235 [==============================] - 166s 16ms/step - loss: 1.0467 - acc: 0.4610 - val_loss: 1.0265 - val_acc: 0.4977
Epoch 4/5
10235/10235 [==============================] - 166s 16ms/step - loss: 1.0398 - acc: 0.4736 - val_loss: 1.0181 - val_acc: 0.4977
Epoch 5/5
1265/1265 [==============================] - 7s 6ms/step
0.4893280636180531
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 178s 17ms/step - loss: 1.0592 - acc: 0.4347 - val_loss: 1.0395 - val_acc: 0.4844
Epoch 2/5
10235/10235 [===========================

1265/1265 [==============================] - 13s 10ms/step
0.5019762849619266
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 244s 24ms/step - loss: 1.0605 - acc: 0.4313 - val_loss: 1.0328 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 195s 19ms/step - loss: 1.0512 - acc: 0.4460 - val_loss: 1.0340 - val_acc: 0.5086
Epoch 3/5
10235/10235 [==============================] - 193s 19ms/step - loss: 1.0407 - acc: 0.4617 - val_loss: 1.0315 - val_acc: 0.4759
Epoch 4/5
10235/10235 [==============================] - 193s 19ms/step - loss: 1.0313 - acc: 0.4839 - val_loss: 1.0155 - val_acc: 0.5086
Epoch 5/5
1265/1265 [==============================] - 13s 10ms/step
0.47984189756302964
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 244s 24ms/step - loss: 1.0606 - acc: 0.4350 - val_loss: 1.0351 - val_acc: 0.4798
Epoch 2/5
10235/10235 [======================

10235/10235 [==============================] - 166s 16ms/step - loss: 1.0602 - acc: 0.4335 - val_loss: 1.0375 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 164s 16ms/step - loss: 1.0528 - acc: 0.4446 - val_loss: 1.0406 - val_acc: 0.4969
Epoch 3/5
10235/10235 [==============================] - 164s 16ms/step - loss: 1.0497 - acc: 0.4501 - val_loss: 1.0334 - val_acc: 0.4969
Epoch 4/5
10235/10235 [==============================] - 164s 16ms/step - loss: 1.0445 - acc: 0.4561 - val_loss: 1.0223 - val_acc: 0.4930
Epoch 5/5
1265/1265 [==============================] - 7s 5ms/step
0.4758893284401875
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 164s 16ms/step - loss: 1.0599 - acc: 0.4326 - val_loss: 1.0410 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 163s 16ms/step - loss: 1.0532 - acc: 0.4432 - val_loss: 1.0369 - val_acc: 0.4930
Epoch 3/5
10235/10235 [========================

10235/10235 [==============================] - 232s 23ms/step - loss: 1.0618 - acc: 0.4287 - val_loss: 1.0341 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 185s 18ms/step - loss: 1.0496 - acc: 0.4510 - val_loss: 1.0295 - val_acc: 0.5070
Epoch 3/5
10235/10235 [==============================] - 184s 18ms/step - loss: 1.0413 - acc: 0.4638 - val_loss: 1.0116 - val_acc: 0.4907
Epoch 4/5
10235/10235 [==============================] - 184s 18ms/step - loss: 1.0340 - acc: 0.4801 - val_loss: 1.0140 - val_acc: 0.4899
Epoch 5/5
1265/1265 [==============================] - 13s 10ms/step
0.5035573126299108
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 237s 23ms/step - loss: 1.0612 - acc: 0.4303 - val_loss: 1.0342 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 185s 18ms/step - loss: 1.0518 - acc: 0.4441 - val_loss: 1.0300 - val_acc: 0.5016
Epoch 3/5
10235/10235 [======================

10235/10235 [==============================] - 197s 19ms/step - loss: 1.0602 - acc: 0.4353 - val_loss: 1.0379 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 170s 17ms/step - loss: 1.0510 - acc: 0.4470 - val_loss: 1.0346 - val_acc: 0.4961
Epoch 3/5
10235/10235 [==============================] - 169s 17ms/step - loss: 1.0440 - acc: 0.4620 - val_loss: 1.0366 - val_acc: 0.4852
Epoch 4/5
10235/10235 [==============================] - 169s 17ms/step - loss: 1.0360 - acc: 0.4770 - val_loss: 1.0125 - val_acc: 0.5055
Epoch 5/5
1265/1265 [==============================] - 10s 8ms/step
0.4450592888673775
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 201s 20ms/step - loss: 1.0599 - acc: 0.4330 - val_loss: 1.0407 - val_acc: 0.4883
Epoch 2/5
10235/10235 [==============================] - 171s 17ms/step - loss: 1.0513 - acc: 0.4523 - val_loss: 1.0345 - val_acc: 0.5055
Epoch 3/5
10235/10235 [=======================

Epoch 2/5
10235/10235 [==============================] - 160s 16ms/step - loss: 1.0546 - acc: 0.4384 - val_loss: 1.0356 - val_acc: 0.4798
Epoch 3/5
10235/10235 [==============================] - 160s 16ms/step - loss: 1.0514 - acc: 0.4426 - val_loss: 1.0310 - val_acc: 0.4798
Epoch 4/5
10235/10235 [==============================] - 160s 16ms/step - loss: 1.0484 - acc: 0.4497 - val_loss: 1.0372 - val_acc: 0.5039
Epoch 5/5
1265/1265 [==============================] - 7s 5ms/step
0.4561264824961485
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 163s 16ms/step - loss: 1.0612 - acc: 0.4323 - val_loss: 1.0380 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 156s 15ms/step - loss: 1.0546 - acc: 0.4398 - val_loss: 1.0362 - val_acc: 0.4798
Epoch 3/5
10235/10235 [==============================] - 156s 15ms/step - loss: 1.0518 - acc: 0.4416 - val_loss: 1.0309 - val_acc: 0.4790
Epoch 4/5
10235/10235 [==============

10235/10235 [==============================] - 178s 17ms/step - loss: 1.0515 - acc: 0.4490 - val_loss: 1.0241 - val_acc: 0.4844
Epoch 3/5
10235/10235 [==============================] - 178s 17ms/step - loss: 1.0419 - acc: 0.4644 - val_loss: 1.0561 - val_acc: 0.4112
Epoch 4/5
10235/10235 [==============================] - 177s 17ms/step - loss: 1.0346 - acc: 0.4739 - val_loss: 1.0182 - val_acc: 0.4961
Epoch 5/5
1265/1265 [==============================] - 12s 9ms/step
0.49723320160458684
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 228s 22ms/step - loss: 1.0596 - acc: 0.4382 - val_loss: 1.0432 - val_acc: 0.4883
Epoch 2/5
10235/10235 [==============================] - 178s 17ms/step - loss: 1.0519 - acc: 0.4474 - val_loss: 1.0255 - val_acc: 0.4829
Epoch 3/5
10235/10235 [==============================] - 178s 17ms/step - loss: 1.0427 - acc: 0.4582 - val_loss: 1.0237 - val_acc: 0.5008
Epoch 4/5
10235/10235 [======================

In [13]:
cnn_flair_rounds

[{5: 0.46877470393425863,
  6: 0.4592885378792352,
  7: 0.4592885378792352,
  8: 0.4727272730571008,
  9: 0.47905138377615586,
  10: 0.4885375497369427,
  11: 0.4869565221160768,
  12: 0.488537549784061,
  13: 0.49090909093265006,
  14: 0.4830039526398474,
  15: 0.4885375497369427,
  16: 0.4893280635709348,
  17: 0.49644268802974534,
  18: 0.4790513835170052,
  19: 0.485375494188942,
  20: 0.493280632693777,
  21: 0.49644268802974534,
  22: 0.48458498056698224,
  23: 0.4877470359029506,
  24: 0.49169960507291105,
  25: 0.5035573125827925,
  26: 0.5035573125827925,
  27: 0.4853754944009743,
  28: 0.4450592888673775,
  29: 0.5003952572939424,
  30: 0.505138340297895,
  31: 0.5067193678716426,
  32: 0.49723320195797405,
  33: 0.5003952571997058,
  34: 0.49723320195797405,
  35: 0.4940711465748874},
 {5: 0.46482213471717987,
  6: 0.4569169963772589,
  7: 0.4687747038871403,
  8: 0.44743083036935377,
  9: 0.4640316208831878,
  10: 0.4877470359500689,
  11: 0.4893280636180531,
  12: 0.475889

In [14]:
traces = cnn_flair_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded Flair dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [3]:
gpt = data.get_gpt()

In [5]:
concatenated_gpt = {
    dataset: [np.concatenate(np.array(statement)) for statement in gpt[dataset].statement]
    for dataset in gpt.keys()
}

In [8]:
%%time

cnn_gpt_rounds = [calculate_round(concatenated_gpt) for round in range(5)]

Instructions for updating:
Colocations handled automatically by placer.
2019-06-14 22:29:10,996 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-06-14 22:29:11,087 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-06-14 22:29:11,181 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

1265/1265 [==============================] - 2s 2ms/step
0.43715415052745654
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 59s 6ms/step - loss: 2.4710 - acc: 0.4020 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 2/5
10235/10235 [==============================] - 58s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 3/5
10235/10235 [==============================] - 57s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 4/5
10235/10235 [==============================] - 57s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
0.3541501976402381
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 59s 6ms/step - loss: 3.6531 - acc: 0.4152 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 2/5
10235/10235 [=============================

1265/1265 [==============================] - 3s 2ms/step
0.43715415052745654
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 63s 6ms/step - loss: 11.3846 - acc: 0.2285 - val_loss: 13.0049 - val_acc: 0.1931
Epoch 2/5
10235/10235 [==============================] - 63s 6ms/step - loss: 12.7890 - acc: 0.2065 - val_loss: 13.0049 - val_acc: 0.1931
Epoch 3/5
10235/10235 [==============================] - 62s 6ms/step - loss: 12.7890 - acc: 0.2065 - val_loss: 13.0049 - val_acc: 0.1931
Epoch 4/5
10235/10235 [==============================] - 61s 6ms/step - loss: 12.7890 - acc: 0.2065 - val_loss: 13.0049 - val_acc: 0.1931
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
0.20869565223281092
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 60s 6ms/step - loss: 7.3226 - acc: 0.4254 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 2/5
10235/10235 [===========================

1265/1265 [==============================] - 3s 3ms/step
0.48142292518389557
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 69s 7ms/step - loss: 1.0719 - acc: 0.4196 - val_loss: 1.0327 - val_acc: 0.4759
Epoch 2/5
10235/10235 [==============================] - 66s 6ms/step - loss: 1.0488 - acc: 0.4509 - val_loss: 1.0265 - val_acc: 0.4836
Epoch 3/5
10235/10235 [==============================] - 67s 7ms/step - loss: 1.0387 - acc: 0.4684 - val_loss: 1.0295 - val_acc: 0.4813
Epoch 4/5
10235/10235 [==============================] - 67s 7ms/step - loss: 1.0326 - acc: 0.4737 - val_loss: 1.0198 - val_acc: 0.4860
Epoch 5/5
1265/1265 [==============================] - 3s 3ms/step
0.474308300725085
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 69s 7ms/step - loss: 1.0676 - acc: 0.4225 - val_loss: 1.0381 - val_acc: 0.4930
Epoch 2/5
10235/10235 [==============================] - 67s

1265/1265 [==============================] - 2s 2ms/step
0.3541501976402381
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 60s 6ms/step - loss: 5.9552 - acc: 0.4212 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 58s 6ms/step - loss: 9.0551 - acc: 0.4382 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 3/5
10235/10235 [==============================] - 58s 6ms/step - loss: 9.0551 - acc: 0.4382 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 4/5
10235/10235 [==============================] - 58s 6ms/step - loss: 9.0551 - acc: 0.4382 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 5/5
1265/1265 [==============================] - 2s 2ms/step
0.43715415052745654
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 61s 6ms/step - loss: 5.6971 - acc: 0.4200 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 59

...

1265/1265 [==============================] - 3s 3ms/step
0.43715415052745654
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 70s 7ms/step - loss: 9.3033 - acc: 0.3595 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 2/5
10235/10235 [==============================] - 64s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 3/5
10235/10235 [==============================] - 64s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 4/5
10235/10235 [==============================] - 64s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
0.3541501976402381
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 72s 7ms/step - loss: 11.6251 - acc: 0.2227 - val_loss: 13.0049 - val_acc: 0.1931
Epoch 2/5
10235/10235 [===========================

10235/10235 [==============================] - 63s 6ms/step - loss: 4.8297 - acc: 0.3839 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 2/5
10235/10235 [==============================] - 57s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 3/5
10235/10235 [==============================] - 57s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 4/5
10235/10235 [==============================] - 57s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
0.3541501976402381
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 63s 6ms/step - loss: 5.0213 - acc: 0.3922 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 2/5
10235/10235 [==============================] - 57s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 3/5
10235/10235 [==========================

10235/10235 [==============================] - 70s 7ms/step - loss: 1.0868 - acc: 0.4197 - val_loss: 1.0372 - val_acc: 0.4696
Epoch 2/5
10235/10235 [==============================] - 62s 6ms/step - loss: 7.2642 - acc: 0.4311 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 3/5
10235/10235 [==============================] - 63s 6ms/step - loss: 9.0551 - acc: 0.4382 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 4/5
10235/10235 [==============================] - 63s 6ms/step - loss: 9.0551 - acc: 0.4382 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 5/5
1265/1265 [==============================] - 3s 2ms/step
0.43715415052745654
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 71s 7ms/step - loss: 1.2140 - acc: 0.4084 - val_loss: 1.0588 - val_acc: 0.4237
Epoch 2/5
10235/10235 [==============================] - 64s 6ms/step - loss: 1.0942 - acc: 0.4333 - val_loss: 1.0270 - val_acc: 0.4907
Epoch 3/5
10235/10235 [==============================] - 6

10235/10235 [==============================] - 74s 7ms/step - loss: 9.2903 - acc: 0.3574 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 2/5
10235/10235 [==============================] - 65s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 3/5
10235/10235 [==============================] - 64s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 4/5
10235/10235 [==============================] - 64s 6ms/step - loss: 10.3921 - acc: 0.3553 - val_loss: 10.8458 - val_acc: 0.3271
Epoch 5/5
1265/1265 [==============================] - 3s 3ms/step
0.3541501976402381
Train on 10235 samples, validate on 1284 samples
Epoch 1/5
10235/10235 [==============================] - 74s 7ms/step - loss: 7.8169 - acc: 0.4332 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 2/5
10235/10235 [==============================] - 63s 6ms/step - loss: 9.0551 - acc: 0.4382 - val_loss: 8.3854 - val_acc: 0.4798
Epoch 3/5
10235/10235 [=============================

CPU times: user 3d 2h 15min 16s, sys: 5h 1min 8s, total: 3d 7h 16min 25s
Wall time: 13h 47min 6s


In [9]:
cnn_gpt_rounds

[{5: 0.43715415052745654,
  6: 0.3541501976402381,
  7: 0.3541501976402381,
  8: 0.43715415052745654,
  9: 0.43715415052745654,
  10: 0.43715415052745654,
  11: 0.43715415052745654,
  12: 0.43715415052745654,
  13: 0.3541501976402381,
  14: 0.43715415052745654,
  15: 0.3541501976402381,
  16: 0.3541501976402381,
  17: 0.20869565223281092,
  18: 0.3541501976402381,
  19: 0.3541501976402381,
  20: 0.20869565223281092,
  21: 0.43715415052745654,
  22: 0.43715415052745654,
  23: 0.20869565223281092,
  24: 0.43715415052745654,
  25: 0.20869565223281092,
  26: 0.20869565223281092,
  27: 0.43715415052745654,
  28: 0.43715415052745654,
  29: 0.3541501976402381,
  30: 0.43715415052745654,
  31: 0.3541501976402381,
  32: 0.48142292518389557,
  33: 0.474308300725085,
  34: 0.46324110709631394,
  35: 0.45375494099417224},
 {5: 0.43715415052745654,
  6: 0.3541501976402381,
  7: 0.43715415052745654,
  8: 0.3541501976402381,
  9: 0.43715415052745654,
  10: 0.3541501976402381,
  11: 0.4371541505274565

In [10]:
traces = cnn_gpt_rounds

# Create traces
def create_scatter(counter):
    acc_dict = traces[counter]
    
    return go.Scatter(
        x = list(acc_dict.keys()),
        y = list(acc_dict.values()),
        mode = 'lines+markers',
        name = 'Round ' + str(counter)
    )

trace_data = [create_scatter(trace) for trace in range(len(traces))]

layout = go.Layout(
    title = 'CNN test set accuracy of padded GPT dataset with variable maximum lengths',
)

fig = go.Figure(data = trace_data, layout = layout)

iplot(fig)

In [17]:
x = list(range(5, 36))
x_rev = x[::-1]

# BERT
bert_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_bert_rounds]))
bert_y = [np.average(row) for row in bert_matrix]
bert_y_upper = [row.max() for row in bert_matrix]
bert_y_lower = [row.min() for row in bert_matrix]
bert_y_lower = bert_y_lower[::-1]

bert1 = go.Scatter(
    x = x + x_rev,
    y = bert_y_upper + bert_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,100,80,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'BERT',
)
bert2 = go.Scatter(
    x = x,
    y = bert_y,
    line = dict(color='rgb(0,100,80)'),
    mode = 'lines+markers',
    name = 'BERT',
)

# ELMo
elmo_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_elmo_rounds]))
elmo_y = [np.average(row) for row in elmo_matrix]
elmo_y_upper = [row.max() for row in elmo_matrix]
elmo_y_lower = [row.min() for row in elmo_matrix]
elmo_y_lower = elmo_y_lower[::-1]

elmo1 = go.Scatter(
    x = x + x_rev,
    y = elmo_y_upper + elmo_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(0,176,246,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'ELMo',
)
elmo2 = go.Scatter(
    x = x,
    y = elmo_y,
    line = dict(color='rgb(0,176,246)'),
    mode = 'lines+markers',
    name = 'ELMo',
)

# Transformer-XL
transformerxl_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_transformerxl_rounds]))
transformerxl_y = [np.average(row) for row in transformerxl_matrix]
transformerxl_y_upper = [row.max() for row in transformerxl_matrix]
transformerxl_y_lower = [row.min() for row in transformerxl_matrix]
transformerxl_y_lower = transformerxl_y_lower[::-1]

transformerxl1 = go.Scatter(
    x = x + x_rev,
    y = transformerxl_y_upper + transformerxl_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(231,107,243,0.2)',
    line = dict(color = 'rgba(255,255,255,0)'),
    showlegend = False,
    name = 'Transformer-XL',
)
transformerxl2 = go.Scatter(
    x = x,
    y = transformerxl_y,
    line = dict(color='rgb(231,107,243)'),
    mode = 'lines+markers',
    name = 'Transformer-XL',
)

# GPT
gpt_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_gpt_rounds]))
gpt_y = [np.average(row) for row in gpt_matrix]
gpt_y_upper = [row.max() for row in gpt_matrix]
gpt_y_lower = [row.min() for row in gpt_matrix]
gpt_y_lower = gpt_y_lower[::-1]

gpt1 = go.Scatter(
    x = x + x_rev,
    y = gpt_y_upper + gpt_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(81, 45, 168, 0.2)',
    line = dict(color = 'rgba(81, 45, 168, 0)'),
    showlegend = False,
    name = 'GPT',
)
gpt2 = go.Scatter(
    x = x,
    y = gpt_y,
    line = dict(color='rgb(81, 45, 168)'),
    mode = 'lines+markers',
    name = 'GPT',
)

# Flair
flair_matrix = np.transpose(np.array([np.array(list(acc_round.values())) for acc_round in cnn_flair_rounds]))
flair_y = [np.average(row) for row in flair_matrix]
flair_y_upper = [row.max() for row in flair_matrix]
flair_y_lower = [row.min() for row in flair_matrix]
flair_y_lower = flair_y_lower[::-1]

flair1 = go.Scatter(
    x = x + x_rev,
    y = flair_y_upper + flair_y_lower,
    fill = 'tozerox',
    fillcolor = 'rgba(205,12,24,0.2)',
    line = dict(color = 'rgba(205,12,24,0)'),
    showlegend = False,
    name = 'Flair',
)
flair2 = go.Scatter(
    x = x,
    y = flair_y,
    line = dict(color='rgb(205,12,24)'),
    mode = 'lines+markers',
    name = 'Flair',
)

# Baseline and highest performance
baseline = go.Scatter(
    x = x,
    y = [0.4428 for i in x],
    line = dict(color='rgb(0,0,0)'),
    mode = 'lines',
    name = 'Majority vote',
)
best_past_performance = go.Scatter(
    x = x,
    y = [0.4903 for i in x],
    mode = 'lines',
    name = 'Khurana (2017)',
)

data = [bert1, bert2, elmo1, elmo2, transformerxl1, transformerxl2, flair1, flair2, gpt1, gpt2, baseline, best_past_performance]
layout = go.Layout(
    title = 'CNN test set accuracy of padded datasets with variable maximum lengths',
    paper_bgcolor = 'rgb(255,255,255)',
    plot_bgcolor = 'rgb(229,229,229)',
    xaxis = dict(
        gridcolor = 'rgb(255,255,255)',
        range = [5,35],
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid = True,
        showline = False,
        showticklabels = True,
        tickcolor = 'rgb(127,127,127)',
        ticks = 'outside',
        zeroline = False
    ),
)

fig = go.Figure(data = data, layout = layout)

iplot(fig)

<hr>

### References

```
@article{DBLP:journals/corr/Wang17j,
  author    = {William Yang Wang},
  title     = {"Liar, Liar Pants on Fire": {A} New Benchmark Dataset for Fake News
               Detection},
  journal   = {CoRR},
  volume    = {abs/1705.00648},
  year      = {2017},
  url       = {http://arxiv.org/abs/1705.00648},
  archivePrefix = {arXiv},
  eprint    = {1705.00648},
  timestamp = {Mon, 13 Aug 2018 16:48:58 +0200},
  biburl    = {https://dblp.org/rec/bib/journals/corr/Wang17j},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}
```